# Porto taxi dataset

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

raw_dataset = pd.read_csv(
    "../assets/taxi_porto.csv",
    usecols=["TIMESTAMP", "POLYLINE", "TAXI_ID", "MISSING_DATA"],
)

WINDOW = 5

index = pd.date_range(
    start = datetime.fromtimestamp(min(raw_dataset["TIMESTAMP"])), 
    end = datetime.fromtimestamp(max(raw_dataset["TIMESTAMP"])), 
    freq = f"{WINDOW}min"
)

In [2]:
raw_dataset = raw_dataset.dropna()

In [3]:
raw_dataset.head()

,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE
0,20000589,1372636858,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,20000596,1372637303,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,20000320,1372636951,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,20000520,1372636854,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,20000337,1372637091,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [4]:
len(raw_dataset["TAXI_ID"].unique())

448

In [5]:
from math import floor

LON_MIN, LON_MAX, LAT_MIN, LAT_MAX = None, None, None, None
with open("../assets/BBox_Porto.txt", "r") as f:
    LON_MIN, LON_MAX, LAT_MIN, LAT_MAX = eval(f.readline())
BBox = (LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)

N_ROWS = 20
N_COLS = 20
N_CELLS = N_ROWS * N_COLS

LAT_DIFF = LAT_MAX - LAT_MIN
LON_DIFF = LON_MAX - LON_MIN

CELL_LON = LON_DIFF / N_COLS
CELL_LAT = LAT_DIFF / N_ROWS

def get_cell(lat: float, lon: float) -> int:
    lon_delta = lon - LON_MIN
    col = floor(lon_delta / CELL_LON)
    
    lat_delta = lat - LAT_MIN
    row = floor(lat_delta / CELL_LAT)
    
    return row*N_COLS + col

In [5]:
gps = raw_dataset["POLYLINE"].apply(eval).dropna()
gps = gps.rename("gps")

KeyboardInterrupt: 

In [ ]:
gps_dataset = pd.DataFrame(
  data={
    "timestamp": raw_dataset["TIMESTAMP"], 
    "gps": gps, 
    "taxi_id": raw_dataset["TAXI_ID"]
  }
)

In [ ]:
df = pd.DataFrame(
    index=index,
    data=np.zeros((len(index), N_CELLS)),
    columns=[f"cell_{i}" for i in range(N_CELLS)]
)

In [ ]:
base_ts = min(raw_dataset["TIMESTAMP"])

for i, g in gps_dataset.iterrows():
    first_ts = g["timestamp"]
    trajectory = g["gps"]
    if isinstance(trajectory, list) and len(trajectory) > 0:
        for j, (lon, lat) in enumerate(trajectory):
            if LON_MIN < lon < LON_MAX and LAT_MIN < lat < LAT_MAX:
                cell = get_cell(lat=lat, lon=lon)
                ts = first_ts + j*15
                
                row = floor((ts - base_ts) / (60 * 15))
                
                df.iloc[row][cell] += 1

In [ ]:
df.head()

In [ ]:
df.to_csv("../assets/porto_taxi_cells.csv")

### Prepare data to map Azure Function Traces

In [6]:
ddd = raw_dataset[
  ~raw_dataset["MISSING_DATA"]
].sort_values(by = "TIMESTAMP")[["TIMESTAMP", "TAXI_ID", "POLYLINE"]]
ddd["second"] = ddd["TIMESTAMP"] - ddd["TIMESTAMP"].min() + 1
ddd["minute"] = np.ceil(ddd["second"] / 60)
ddd["day"] = np.ceil(ddd["second"] / 3600 / 24)
ddd["gps"] = ddd["POLYLINE"].apply(eval)
ddd

,TIMESTAMP,TAXI_ID,POLYLINE,second,minute,day,gps
577,1372636853,20000380,"[[-8.610291,41.140746],[-8.6103,41.140755],[-8...",1,1.0,1.0,"[[-8.610291, 41.140746], [-8.6103, 41.140755],..."
3,1372636854,20000520,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",2,1.0,1.0,"[[-8.574678, 41.151951], [-8.574705, 41.151942..."
0,1372636858,20000589,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",6,1.0,1.0,"[[-8.618643, 41.141412], [-8.618499, 41.141376..."
10,1372636875,20000233,"[[-8.619894,41.148009],[-8.620164,41.14773],[-...",23,1.0,1.0,"[[-8.619894, 41.148009], [-8.620164, 41.14773]..."
64,1372636896,20000360,"[[-8.617599,41.146137],[-8.617581,41.14593],[-...",44,1.0,1.0,"[[-8.617599, 41.146137], [-8.617581, 41.14593]..."
...,...,...,...,...,...,...,...
1710412,1404172716,20000359,"[[-8.599248,41.149197],[-8.598645,41.148504],[...",31535864,525598.0,365.0,"[[-8.599248, 41.149197], [-8.598645, 41.148504..."
1710511,1404172718,20000066,"[[-8.601192,41.181741],[-8.601201,41.181741],[...",31535866,525598.0,365.0,"[[-8.601192, 41.181741], [-8.601201, 41.181741..."
1710603,1404172736,20000483,"[[-8.58609,41.175504],[-8.586351,41.174955],[-...",31535884,525599.0,365.0,"[[-8.58609, 41.175504], [-8.586351, 41.174955]..."
1710532,1404172754,20000518,"[[-8.604765,41.16123],[-8.604792,41.161167]]",31535902,525599.0,365.0,"[[-8.604765, 41.16123], [-8.604792, 41.161167]]"


In [7]:
# extract the first 14 days
day14 = ddd[ddd["day"] <= 14]
day14.index = range(len(day14))
day14

,TIMESTAMP,TAXI_ID,POLYLINE,second,minute,day,gps
0,1372636853,20000380,"[[-8.610291,41.140746],[-8.6103,41.140755],[-8...",1,1.0,1.0,"[[-8.610291, 41.140746], [-8.6103, 41.140755],..."
1,1372636854,20000520,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",2,1.0,1.0,"[[-8.574678, 41.151951], [-8.574705, 41.151942..."
2,1372636858,20000589,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",6,1.0,1.0,"[[-8.618643, 41.141412], [-8.618499, 41.141376..."
3,1372636875,20000233,"[[-8.619894,41.148009],[-8.620164,41.14773],[-...",23,1.0,1.0,"[[-8.619894, 41.148009], [-8.620164, 41.14773]..."
4,1372636896,20000360,"[[-8.617599,41.146137],[-8.617581,41.14593],[-...",44,1.0,1.0,"[[-8.617599, 41.146137], [-8.617581, 41.14593]..."
...,...,...,...,...,...,...,...
68140,1373846381,20000199,"[[-8.585631,41.148522],[-8.585613,41.148576],[...",1209529,20159.0,14.0,"[[-8.585631, 41.148522], [-8.585613, 41.148576..."
68141,1373846396,20000305,"[[-8.606493,41.144634],[-8.607186,41.144544],[...",1209544,20160.0,14.0,"[[-8.606493, 41.144634], [-8.607186, 41.144544..."
68142,1373846423,20000657,"[[-8.689356,41.168115],[-8.688564,41.167296],[...",1209571,20160.0,14.0,"[[-8.689356, 41.168115], [-8.688564, 41.167296..."
68143,1373846436,20000625,"[[-8.585676,41.148423],[-8.585703,41.148657],[...",1209584,20160.0,14.0,"[[-8.585676, 41.148423], [-8.585703, 41.148657..."


In [8]:
# compute taxis full paths
full_taxi_path = pd.DataFrame()
for taxi_id, taxi_data in day14.groupby("TAXI_ID"):
  taxi_info = {"second": [], "cell": []}
  for idx, (ts, _, _, second, minute, day, trajectory) in taxi_data.iterrows():
    # add information to the full path
    if isinstance(trajectory, list) and len(trajectory) > 0:
      # convert the trajectory in a list of cells
      cells = []
      times = []
      for j, (lon, lat) in enumerate(trajectory):
        if LON_MIN < lon < LON_MAX and LAT_MIN < lat < LAT_MAX:
          cells.append(get_cell(lat=lat, lon=lon))
          times.append(second + j)
      taxi_info["cell"] += cells
      taxi_info["second"] += times
  # add taxi id and merge
  taxi_info = pd.DataFrame(taxi_info)
  taxi_info["taxi_id"] = [str(taxi_id)] * len(taxi_info)
  full_taxi_path = pd.concat([full_taxi_path, taxi_info], ignore_index = True)

full_taxi_path.sort_values(by = "second", inplace = True)
full_taxi_path.to_csv("../data/full_taxi_path.csv", index = False)
full_taxi_path

,second,cell,taxi_id
846723,1.0,49.0,20000380
846724,2.0,49.0,20000380
846725,3.0,49.0,20000380
846726,4.0,48.0,20000380
846727,5.0,48.0,20000380
...,...,...,...
1131089,1209632.0,90.0,20000517
1131090,1209633.0,90.0,20000517
1131091,1209634.0,90.0,20000517
1131092,1209635.0,90.0,20000517


### Assignment to nodes

In [31]:
from pathlib import Path

n_nodes = 20
k=19
n_simulations = 10
network_path = Path(f"data/networks/porto_{n_nodes}n_{k}k")
output_path = Path(f"data/datasets/porto_{n_nodes}n_{k}k")

In [32]:
from category_encoders import BinaryEncoder
import pandas as pd

df = pd.read_csv("assets/porto_taxi_cells.csv", index_col=0, parse_dates=True)

def encode_time(df: pd.DataFrame) -> pd.DataFrame:
    df["hour"] = df.index.hour
    df["weekday"] = df.index.weekday
    return BinaryEncoder(cols=["hour", "weekday"]).fit_transform(df)

In [33]:
for n in range(n_simulations):
    dataset_path = output_path / str(n)
    dataset_path.mkdir(exist_ok=True, parents=True)
    
    towers = pd.read_csv(network_path / str(n) / "towers.csv")

    tower_cells = [
        [] for i in range(n_nodes)
    ]
    
    for i in range(N_ROWS):
        for j in range(N_COLS):
            cell = i*N_ROWS + j
            
            lat_begin = LAT_MIN + i*CELL_LAT
            lon_begin = LON_MIN + j*CELL_LON
            
            lat_center = lat_begin + CELL_LAT / 2
            lon_center = lon_begin + CELL_LON / 2
            
            min_dist = None
            nearest_tower = None
            for n_tower, tower in towers.iterrows():
                dist = (lat_center - tower["lat"])**2 + (lon_center - tower["lon"])**2
                if min_dist is None or min_dist > dist:
                    min_dist = dist
                    nearest_tower = n_tower
                    
            tower_cells[nearest_tower].append(cell)
            
    tower_calls = pd.DataFrame(
        np.zeros((len(index), n_nodes)), 
        columns=[f"tower_{i}" for i in range(n_nodes)], 
        index=index
    )
    
    for i, cells in enumerate(tower_cells):
        ind = f"tower_{i}"
        for cell in cells:
            tower_calls[ind] += df[f"cell_{cell}"]
            
        # tower_calls[ind] += 1
        # tmp_ind = tower_calls[ind][1:].index
        # tower_calls[ind] = pd.Series(
        #     tower_calls[ind][1:].to_numpy() / tower_calls[ind][:-1].to_numpy(),
        #     index=tmp_ind,
        # )
    
                        
    quantiles = tower_calls.quantile(0.8)
    tower_calls = tower_calls.clip(0, quantiles, axis=1)

    # max_value = tower_calls.max().max()
    # if max_value > 0:
    #     tower_calls = tower_calls / max_value
    # else:
    #     print("Max value is 0!!")
    
    node_datasets = [
        encode_time(pd.DataFrame({"requests": tower_calls[col]}))
        for col in tower_calls.columns
    ]
    
    for i, ds in enumerate(node_datasets):
        ds = ds.dropna()
        ds.to_parquet(dataset_path / f"node_{i}_notscaled.parquet")
    

/Users/personal/Documents/thesis-workspace/gossip_learning_simulator/.venv/lib/python3.11/site-packages/category_encoders/utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
/Users/personal/Documents/thesis-workspace/gossip_learning_simulator/.venv/lib/python3.11/site-packages/category_encoders/utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
/Users/personal/Documents/thesis-workspace/gossip_learning_simulator/.venv/lib/python3.11/site-packages/category_encoders/utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
/Users/personal/Documents/thes

In [34]:
from utils.data import prepare_dataset_for_training

ds_name = "4in_notscaled"

for n in range(n_simulations):
    towers_datasets = [
        pd.read_parquet(output_path / str(n) / f"node_{i}_notscaled.parquet") for i in range(n_nodes)
    ]
    
    small_ds_nodes = list(range(10))
    
    start_i = round(len(towers_datasets[0]) * 0.8)
    start_ind = index[start_i]
    
    towers_datasets = [
        ts[start_ind:] 
        if i in small_ds_nodes
        else ts
        for i, ts in enumerate(towers_datasets)
    ]
    
    # scale
    # max_value = max(*[ts['requests'].max() for ts in towers_datasets])
    # for i in range(len(towers_datasets)):
    #     towers_datasets[i]["requests"] /= max_value
    # 
    # folder = output_path / str(n) / ds_name
    # folder.mkdir(parents=True, exist_ok=True)
    # with (folder / "scaling_factor.txt").open("w") as f:
    #     f.write(str(max_value))
    
    prepare_dataset_for_training(
        towers_datasets=towers_datasets,
        output_folder=output_path / str(n) / ds_name,
        input_timesteps=4,
        output_timesteps=1,
        n_functions=1,
        n_auxiliary_features=8,
    )

Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
